## Progetto : Protezione e Filtro da Commenti Dannosi

**Descrizione**

Questo progetto mira a creare un ambiente online più sicuro e rispettoso attraverso l'identificazione e la filtrazione di commenti potenzialmente dannosi. Utilizzando tecniche di Deep Learning avanzate, sviluppiamo un modello in grado di analizzare i commenti degli utenti in tempo reale, classificandoli in base al grado di dannosità del linguaggio. Questo sistema multilabel di classificazione fornisce una valutazione dettagliata del contenuto testuale, contribuendo significativamente alla protezione da linguaggi tossici, osceni, minacciosi, insultanti e discriminatori.
<br><br>
**Punti Chiave del Progetto**

1. Preprocessamento del Testo: Implementazione di tecniche avanzate per eliminare token non significativi, migliorando l'accuratezza della classificazione rimuovendo elementi che non contribuiscono al significato semantico del testo.

2. Trasformazione in Sequenze: Conversione del testo preprocessato in sequenze, preparando i dati per un'analisi approfondita che cattura le dinamiche e le relazioni semantiche all'interno del corpus testuale.
  
3. Modello di Deep Learning: Costruzione di un modello con layer ricorrenti, ottimizzato per il task di classificazione multilabel, in grado di interpretare la complessità e le sfumature del linguaggio naturale.
   
4. Classificazione in Tempo Reale: In fase di predizione, il modello valuta i commenti fornendo un vettore di classificazione: un commento viene considerato non dannoso se il vettore risultante è composto da zeri [0,0,0,0,0,0], altrimenti, la presenza di almeno un "1" indica la categoria di dannosità rilevata.


# 1. Il Dataset

Il progetto utilizza un dataset, disponibile in un file CSV, che include una raccolta di commenti. Ogni commento nel dataset è associato a una valutazione che ne indica il livello di dannosità. <br>Inizieremo scaricando questo dataset per poi procedere con la sua elaborazione.

In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
import pandas as pd
BASE_URL = "https://s3.eu-west-3.amazonaws.com/profession.ai/datasets/"
df = pd.read_csv(BASE_URL+"Filter_Toxic_Comments_dataset.csv")

In [3]:
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,sum_injurious
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0


Nel dataset, troviamo una gamma di commenti che varia dai contenuti esplicitamente negativi a quelli neutrali o completamente privi di elementi dannosi. Importante per la nostra analisi è la presenza di un'etichetta per ogni commento: questa etichetta assume valore 1 per indicare un commento dannoso in relazione alla specifica categoria di dannosità menzionata nella colonna corrispondente. In aggiunta, il dataset include una colonna denominata "sum_injurious", la quale rappresenta la somma delle valutazioni di dannosità attribuite a ogni commento, offrendo una panoramica complessiva del livello di negatività espresso.

Se desideriamo individuare i commenti che presentano due specifiche tipologie di dannosità, possiamo eseguire un'operazione di filtraggio sui dati. <br>Questo ci permette di selezionare solo quei commenti che soddisfano il nostro criterio specifico, ovvero avere due valutazioni di dannosità quindi ad esempio:

In [4]:
df[df['sum_injurious']==2]

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,sum_injurious
51,GET FUCKED UP. GET FUCKEEED UP. GOT A DRINK T...,1,0,1,0,0,0,2
58,My Band Page's deletion. You thought I was gon...,1,0,1,0,0,0,2
79,Hi! I am back again!\nLast warning!\nStop undo...,1,0,0,1,0,0,2
86,"Would you both shut up, you don't run wikipedi...",1,0,0,0,1,0,2
168,"You should be fired, you're a moronic wimp who...",1,0,0,0,1,0,2
...,...,...,...,...,...,...,...,...
159253,what do you mean \n\nwhy don't you keep your n...,1,0,1,0,0,0,2
159334,"Horse's ass \n\nSeriously, dude, what's that h...",1,0,1,0,0,0,2
159449,I think he is a gay fag!!!,1,0,0,0,0,1,2
159514,YOU ARE A MISCHIEVIOUS PUBIC HAIR,1,0,0,0,1,0,2


Il totale dei commenti sono :

In [5]:
len(df)

159571

# 2. Preprocessing del Testo

Inizieremo a preparare il testo contenuto nei commenti, rimuovendo gli elementi che non aggiungono valore significativo al significato complessivo del testo. Questo processo di pulizia è fondamentale per aiutare la Rete Neurale a imparare in modo più efficace, concentrandosi solo sulle parti del testo che contribuiscono realmente al contesto semantico.

Per fare questo andremo per step nello specifico eliminando innanzitutto le parole chiamate "stop words", effettuando la Lemmatizzazione, ed infine la rimozione di caratteri speciali e punteggiatura. 

## 2a. Rimozione delle Stop Words

Procediamo ora alla rimozione di eventuali "Stop Words" ovvero parole molto comuni nella lingua considerata, come articoli, preposizioni, congiunzioni, e alcune forme verbali ausiliarie, che non aggiungono significato significativo al testo per molti task di Natural Language Processing (NLP). L'idea è che, rimuovendo queste parole, sia possibile concentrarsi sulle parole che hanno più importanza e significato nel contesto di un'analisi specifica, come l'analisi del sentimento, la classificazione dei documenti, il riconoscimento di entità nominate, ecc.

Per esempio, in inglese, parole come "the", "is", "at", "which", e "on" sono spesso considerate stop words, dato che compaiono molto frequentemente e di solito non portano informazioni critiche per comprendere il significato di un testo.

Per procedere con questo step andiamo a utilizzare la libreria "nltk" utile a questo scopo. <br>
Con il codice sotto abbiamo ricevuto grazie a questa libreria un elenco di parole che possiamo rimuovere dai nostri commenti e che fanno riferimento alle Stop Words :

In [6]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Carica le stop words predefinite per l'inglese
stop_words = set(stopwords.words('english'))
stop_words

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/eugenix/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

Procediamo dunque con l'eliminazione dalla lista:

In [7]:
from nltk.tokenize import word_tokenize

# Definiamo un metodo che pulisce le parole
def delete_stop(text):
    tokens = word_tokenize(text) # Tokenizziamo la frase
    word_filter = [elem for elem in tokens if elem.lower() not in stop_words] #Filtriamo i token che rappresentano le Stopwords
    word_filter_sentence = ' '.join(word_filter) # Ricompongo in unica frase i token rimasti.
    return word_filter_sentence
    
# Applichiamo tale metodo in tutto il dataframe

df['comment_text_filter'] = df['comment_text'].apply(delete_stop)
df

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,sum_injurious,comment_text_filter
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0,Explanation edits made username Hardcore Metal...
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0,D'aww ! matches background colour 'm seemingly...
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0,"Hey man , 'm really trying edit war . 's guy c..."
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0,`` ca n't make real suggestions improvement - ...
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0,", sir , hero . chance remember page 's ?"
...,...,...,...,...,...,...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,0,"`` : : : : : second time asking , view complet..."
159567,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0,0,ashamed horrible thing put talk page . 128.61....
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0,0,"Spitzer Umm , theres actual article prostituti..."
159569,And it looks like it was actually you who put ...,0,0,0,0,0,0,0,looks like actually put speedy first version d...


## 2b. La Lemmatizzazione

Una volta effettuata la rimozione delle stop words, procederemo ad effettuare un ulteriore passaggio di Data Cleaning chiamato "Lemmatizzazione".<br>
La lemmatizzazione è il processo di riduzione delle parole a una forma base o lemma, tenendo conto del loro significato e della loro funzione grammaticale nella frase. <br>
Per far questo utilizzeremo una libreria di Python chiamata "nltk":

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Assicurati di aver scaricato le risorse necessarie da NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Inizializza il lemmatizzatore
lemmatizer = WordNetLemmatizer()


# Applica la lemmatizzazione ai token 
def lemmatize_text(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(w.lower()) for w in tokens]
    
    # Ricrea la frase unendo i token lemmatizzati
    lemmatized_sentence = ' '.join(lemmatized_tokens)
    return lemmatized_sentence
    
df['comment_text_filter_lemmatized'] = df['comment_text_filter'].apply(lemmatize_text)
df


[nltk_data] Downloading package punkt to /home/eugenix/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/eugenix/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/eugenix/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,sum_injurious,comment_text_filter,comment_text_filter_lemmatized
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0,Explanation edits made username Hardcore Metal...,explanation edits made username hardcore metal...
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0,D'aww ! matches background colour 'm seemingly...,d'aww ! match background colour 'm seemingly s...
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0,"Hey man , 'm really trying edit war . 's guy c...","hey man , 'm really trying edit war . 's guy c..."
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0,`` ca n't make real suggestions improvement - ...,`` ca n't make real suggestion improvement - w...
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0,", sir , hero . chance remember page 's ?",", sir , hero . chance remember page 's ?"
...,...,...,...,...,...,...,...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,0,"`` : : : : : second time asking , view complet...","`` : : : : : second time asking , view complet..."
159567,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0,0,ashamed horrible thing put talk page . 128.61....,ashamed horrible thing put talk page . 128.61....
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0,0,"Spitzer Umm , theres actual article prostituti...","spitzer umm , there actual article prostitutio..."
159569,And it looks like it was actually you who put ...,0,0,0,0,0,0,0,looks like actually put speedy first version d...,look like actually put speedy first version de...


## 2c. Rimozione caratteri speciali e punteggiatura

Per il nostro progetto, svilupperemo una funzione specifica che si occupa di rimuovere la punteggiatura e i caratteri speciali dal testo.

In [9]:
import string

# Definizione della funzione per rimuovere la punteggiatura da una stringa di testo
def remove_punctuation(text):
    # Usa una comprehension list per mantenere solo i caratteri non presenti in string.punctuation
    token_no_punct = [char for char in text if char not in string.punctuation]
    # Unisce i caratteri rimanenti in una stringa
    words_wo_punct = ''.join(token_no_punct)
    return words_wo_punct

Andiamo ad eseguirla sui nostri commenti :

In [10]:
df['comment_text_filter_lemmatized'] = df['comment_text_filter_lemmatized'].apply(remove_punctuation)
df

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,sum_injurious,comment_text_filter,comment_text_filter_lemmatized
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0,Explanation edits made username Hardcore Metal...,explanation edits made username hardcore metal...
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0,D'aww ! matches background colour 'm seemingly...,daww match background colour m seemingly stuc...
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0,"Hey man , 'm really trying edit war . 's guy c...",hey man m really trying edit war s guy const...
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0,`` ca n't make real suggestions improvement - ...,ca nt make real suggestion improvement wonde...
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0,", sir , hero . chance remember page 's ?",sir hero chance remember page s
...,...,...,...,...,...,...,...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,0,"`` : : : : : second time asking , view complet...",second time asking view completely cont...
159567,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0,0,ashamed horrible thing put talk page . 128.61....,ashamed horrible thing put talk page 128611993
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0,0,"Spitzer Umm , theres actual article prostituti...",spitzer umm there actual article prostitution...
159569,And it looks like it was actually you who put ...,0,0,0,0,0,0,0,looks like actually put speedy first version d...,look like actually put speedy first version de...


# 3. Trasformazione in Sequenze

Una volta completato il preprocessing dei dati, possiamo trasformare le liste di token in sequenze numeriche tramite il Tokenizer di Keras. Questa operazione ci permette di preparare i dati in modo adeguato per l'addestramento di modelli di deep learning, i quali necessitano di input in forma numerica.

Prima pero' di procedere andiamo a segmentare il dataset in componente train e componente di test:

In [11]:
from sklearn.model_selection import train_test_split

# Separazione delle features e delle etichette
X = df['comment_text_filter_lemmatized'].tolist()
y = df[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]

# Divisione in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Ora, trasformeremo `x_train` e `x_test` in sequenze numeriche, sfruttando il Tokenizer di Keras precedentemente addestrato sui dati contenuti in `x_train`:

In [12]:
from keras.preprocessing.text import Tokenizer

# Inizializza il Tokenizer
# num_words è il numero massimo di parole da tenere, basato sulla frequenza delle parole.
# Ogni parola riceverà un ID univoco. Solo le parole più comuni num_words verranno mantenute.
tokenizer = Tokenizer(num_words=10000)

# Addestra il Tokenizer sulle liste di token del train
tokenizer.fit_on_texts(X_train)

# Converti le liste di token in sequenze numeriche (per test e train)
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test  =  tokenizer.texts_to_sequences(X_test)

# Stampa solo le prime 5 sequenze per verificare
print(sequences_train[:5])


[[7499, 3114, 3799, 7499, 3799, 267, 7499, 964, 1749, 696], [1034, 23, 729, 104, 7, 6671, 1290, 334, 454, 3661, 6069, 7, 1595, 2496, 273, 610, 169, 7309, 33, 494, 7, 148, 1152, 183, 860, 50, 2048, 4722, 17, 19, 1, 88, 1479, 8080, 69, 81, 56, 539, 9526], [5081, 57, 6594, 6002, 176, 2814, 811, 42, 5630, 1430, 220, 140, 1338, 3248, 4653, 3495, 2461, 637, 1338, 1725, 2156, 5630, 5837, 6521, 2377, 1283, 6445, 1962, 617, 3248, 4653, 3495, 2461, 4823, 389, 6070, 583, 1962, 396, 637, 1338, 1725, 316, 5630, 349, 1962, 155, 57, 922, 474, 60, 2168, 2014, 879, 1233, 5081, 617, 262, 382, 2344, 51, 814, 237, 1962, 1622, 5035, 57, 498, 154, 1826, 35, 5197, 1551, 42, 317, 312, 1430, 353, 6905, 6445, 1962, 474, 6192, 94, 6975, 5630, 1962, 120, 129, 9384, 8636, 1, 57, 240, 770, 2469, 9220, 6975, 299, 23, 2278, 2250, 164, 1129, 5837, 2377, 1962, 929, 48, 2344, 51, 10, 814, 237, 1962, 131, 5837, 353, 389, 804, 4125, 1980, 396, 617, 1129, 5837, 454, 1399, 1085, 929, 48, 1261, 12, 5890, 4393, 91, 546, 498, 

Andiamo a definirci la dimensione del vocabolario ottenuto tramite il Tokenizer attraverso la len() :

In [13]:
vocabulary_size = len(tokenizer.word_index)+1
vocabulary_size

193930

Adesso procederemo con la standardizzazione delle sequenze generate per i dati di addestramento e test, applicando il **"Padding"**. <br>Questo processo uniformerà la lunghezza di tutte le sequenze, impostandola pari alla lunghezza massima trovata nelle sequenze di addestramento.

In [14]:
from keras.preprocessing.sequence import pad_sequences

#Mi elaboro la lunghezza delle sequence train (e le uso anche per il test come fatto per tokenizer prima)
maxlen = max(sequences_train, key=len) 
len_max_len = len(maxlen)

padded_sequences_train = pad_sequences(sequences_train, maxlen = len_max_len, padding='pre')  
padded_sequences_test = pad_sequences(sequences_test, maxlen = len_max_len, padding='pre')  


# 4. Implementazione Rete RNN 

Come abbiamo visto nel dataframe noi abbiamo un tot. di categorie che classificano la frase rivediamolo:

In [15]:
df[['comment_text','toxic','severe_toxic','obscene','threat','insult','identity_hate']]

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,And it looks like it was actually you who put ...,0,0,0,0,0,0


Essendoci sei categorie che definiscono la classificazione dei commenti, la nostra Rete Neurale prevederà un output di sei neuroni. Per il layer di output, utilizzeremo la funzione di attivazione Softmax, che è adatta per la classificazione multiclasse.

Integreremo quindi uno strato di Embedding nella struttura della nostra rete, impostando come dimensione di input il "vocabulary size" ottenuto dal nostro tokenizer. Aggiungeremo poi uno strato LSTM, elemento chiave della nostra Rete Neurale Ricorrente, che elaborerà sequenzialmente l'intero insieme di dati. Concluderemo con uno strato Dense di output, utilizzando la funzione di attivazione softmax, per classificare i dati nelle sei categorie specifiche.

In [16]:
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [17]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [18]:
model_LSTM = Sequential()
model_LSTM.add(Embedding(vocabulary_size, 128, input_length=len_max_len))
model_LSTM.add(LSTM(64, activation='tanh'))
model_LSTM.add(Dense(6, activation='softmax'))
model_LSTM.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1250, 128)         24823040  
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 6)                 390       
                                                                 
Total params: 24872838 (94.88 MB)
Trainable params: 24872838 (94.88 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Abbiamo definito quindi un architettura fatta da un Layer Embedding che riceverà in ingresso un numero pari al numero di parole definite nel dizionario del tokenizer addestrato sul dataset di train.

Nel Layer successivo andiamo a definire la nostra Rete neurale ricorrente tramite il Layer LSTM che dispone di 64 neuroni.

Infine il layer Denso che mi consente la multi classificazione nelle 6 classi previste.

Andiamo ora a compilare e ad addestrare la nostra Rete Neurale Ricorrente:

In [19]:
model_LSTM.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])s

In [20]:
model_LSTM.fit(padded_sequences_train, y_train, validation_split=0.2, epochs=3, batch_size=32)

Epoch 1/3
3192/3192 [==============================] - 1845s 578ms/step - loss: 0.0913 - accuracy: 0.9901 - val_loss: 0.0597 - val_accuracy: 0.9943
Epoch 2/3
3192/3192 [==============================] - 1839s 576ms/step - loss: 0.0537 - accuracy: 0.9941 - val_loss: 0.0546 - val_accuracy: 0.9943
Epoch 3/3
3192/3192 [==============================] - 1839s 576ms/step - loss: 0.0508 - accuracy: 0.9941 - val_loss: 0.0533 - val_accuracy: 0.9943


Procediamo infine a fornire le metriche di loss e accuracy finali tramite il comando evaluate:

In [23]:
model_LSTM.evaluate(padded_sequences_test, y_test)

998/998 [==============================] - 146s 146ms/step - loss: 0.0518 - accuracy: 0.9941


[0.05177713930606842, 0.9941093325614929]